# 2.2
I am using "The Pit and the Pendulum" by Edgar Allan Poe (which is in the public domain) for text preprocessing to get some different results from using "The Verdict" by Edith Wharton.

In [1]:
with open("pit-and-pendulum.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print(f"total number of character: {len(raw_text)}")
print(raw_text[:99])

total number of character: 33679
I was sick—sick unto death with that long agony; and when they at length unbound me, and I was perm


In [2]:
# Split a text on whitespace characters
import re
text = "Welcome, everyone. My name, is Cheng Guo."
result = re.split(r'(\s)', text)
print(result)

['Welcome,', ' ', 'everyone.', ' ', 'My', ' ', 'name,', ' ', 'is', ' ', 'Cheng', ' ', 'Guo.']


In [3]:
# Split on whitespaces, commas, and periods
result = re.split(r'([,.]|\s)', text)
print(result)

['Welcome', ',', '', ' ', 'everyone', '.', '', ' ', 'My', ' ', 'name', ',', '', ' ', 'is', ' ', 'Cheng', ' ', 'Guo', '.', '']


In [4]:
# Remove redundant whitespace characters
result = [item for item in result if item.strip()]
print(result)

['Welcome', ',', 'everyone', '.', 'My', 'name', ',', 'is', 'Cheng', 'Guo', '.']


In [5]:
# Adjust to handle other types of punctuation
text = "Wow, everyone! Am I Cheng--Guo?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Wow', ',', 'everyone', '!', 'Am', 'I', 'Cheng', '--', 'Guo', '?']


In [6]:
# PRocessing the short story
preprocessed = re.split(r'([,.:;?_!"()\']|—|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

7061
['I', 'was', 'sick', '—', 'sick', 'unto', 'death', 'with', 'that', 'long', 'agony', ';', 'and', 'when', 'they', 'at', 'length', 'unbound', 'me', ',', 'and', 'I', 'was', 'permitted', 'to', 'sit', ',', 'I', 'felt', 'that']
